In [1]:
import streamlit as st
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Make sure VADER is downloaded
nltk.download('vader_lexicon')

# Load saved components
@st.cache_resource
def load_data():
    df = pd.read_csv("Movies.csv")
    with open("tfidf_vectorizer.pkl", "rb") as f:
        tfidf_vectorizer = pickle.load(f)
    with open("cosine_similarity.pkl", "rb") as f:
        cosine_sim = pickle.load(f)
    return df, tfidf_vectorizer, cosine_sim

df, tfidf_vectorizer, cosine_sim = load_data()

# Sentiment Analyzer
sentiment = SentimentIntensityAnalyzer()

# Recommendation function
def get_recommendations(title, df, cosine_sim):
    title = title.lower()
    if title not in df['title'].str.lower().values:
        return None

    idx = df[df['title'].str.lower() == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 5

    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices].values

# Sentiment on input (for fun/demo)
def get_sentiment(text):
    score = sentiment.polarity_scores(text)['compound']
    if score >= 0.05:
        return "Positive 😊"
    elif score <= -0.05:
        return "Negative 😞"
    else:
        return "Neutral 😐"

# UI
st.title("🎬 Movie Recommendation System + Sentiment")

movie_input = st.text_input("Enter a movie you like", "")

if st.button("Recommend"):
    if movie_input.strip() == "":
        st.warning("Please enter a movie name.")
    else:
        recommendations = get_recommendations(movie_input, df, cosine_sim)
        if recommendations is None:
            st.error("Movie not found in database.")
        else:
            st.success(f"Top 5 recommendations for: {movie_input}")
            for rec in recommendations:
                st.write(f"👉 {rec}")

st.markdown("---")
st.subheader("🧠 Test the Sentiment of Your Text")
user_review = st.text_area("Enter a review or comment", "")

if st.button("Analyze Sentiment"):
    if user_review.strip() == "":
        st.warning("Please enter some text.")
    else:
        sent = get_sentiment(user_review)
        st.info(f"Sentiment: **{sent}**")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
2025-06-26 22:21:16.258 
  command:

    streamlit run C:\Users\DELL\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-26 22:21:16.811 Session state does not function when running a script without `streamlit run`
